In [1]:
!pip install opendatasets

In [2]:
!pip install contractions

In [18]:
import numpy as np                                      # Import numpy.
import pandas as pd                                     # Import pandas.
import nltk                                             # Import Natural Language Tool-Kit.
import opendatasets as od

import re, string, unicodedata                          # Import Regex, string and unicodedata.
import contractions                                     # Import contractions library.
from bs4 import BeautifulSoup 

nltk.download('stopwords')                              # Download Stopwords.
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn.metrics import classification_report
from sklearn import metrics

from nltk.corpus import stopwords                       # Import stopwords.
from nltk.tokenize import word_tokenize, sent_tokenize  # Import Tokenizer.
from nltk.stem.wordnet import WordNetLemmatizer         # Import Lemmatizer.
from nltk import word_tokenize, pos_tag, pos_tag_sents  # Import pos_tag

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/manisha/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/manisha/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/manisha/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/manisha/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# 1. Download the file and set it as a Dataframe.

In [4]:
# Loading data into pandas dataframe
od.download("https://www.kaggle.com/andrewmvd/trip-advisor-hotel-reviews")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username:

  manisha9608


Your Kaggle Key:

  ································


 19%|█▉        | 1.00M/5.14M [00:00<00:00, 6.29MB/s]

100%|██████████| 5.14M/5.14M [00:00<00:00, 13.5MB/s]


In [6]:
data = pd.read_csv("trip-advisor-hotel-reviews/tripadvisor_hotel_reviews.csv")

In [7]:
data.shape 

(20491, 2)

In [8]:
data.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [9]:
#Check for null entries
data.isnull().sum(axis=0)

Review    0
Rating    0
dtype: int64

In [10]:
pd.set_option('display.max_colwidth', None) # Display full dataframe information (Non-turncated Text column.)
data.head()   

,Review,Rating
0,"nice hotel expensive parking got good deal stay hotel anniversary, arrived late evening took advice previous reviews did valet parking, check quick easy, little disappointed non-existent view room room clean nice size, bed comfortable woke stiff neck high pillows, not soundproof like heard music room night morning loud bangs doors opening closing hear people talking hallway, maybe just noisy neighbors, aveda bath products nice, did not goldfish stay nice touch taken advantage staying longer, location great walking distance shopping, overall nice experience having pay 40 parking night,",4
1,"ok nothing special charge diamond member hilton decided chain shot 20th anniversary seattle, start booked suite paid extra website description not, suite bedroom bathroom standard hotel room, took printed reservation desk showed said things like tv couch ect desk clerk told oh mixed suites description kimpton website sorry free breakfast, got kidding, embassy suits sitting room bathroom bedroom unlike kimpton calls suite, 5 day stay offer correct false advertising, send kimpton preferred guest website email asking failure provide suite advertised website reservation description furnished hard copy reservation printout website desk manager duty did not reply solution, send email trip guest survey did not follow email mail, guess tell concerned guest.the staff ranged indifferent not helpful, asked desk good breakfast spots neighborhood hood told no hotels, gee best breakfast spots seattle 1/2 block away convenient hotel does not know exist, arrived late night 11 pm inside run bellman busy chating cell phone help bags.prior arrival emailed hotel inform 20th anniversary half really picky wanted make sure good, got nice email saying like deliver bottle champagne chocolate covered strawberries room arrival celebrate, told needed foam pillows, arrival no champagne strawberries no foam pillows great room view alley high rise building good not better housekeeping staff cleaner room property, impressed left morning shopping room got short trips 2 hours, beds comfortable.not good ac-heat control 4 x 4 inch screen bring green shine directly eyes night, light sensitive tape controls.this not 4 start hotel clean business hotel super high rates, better chain hotels seattle,",2
2,"nice rooms not 4* experience hotel monaco seattle good hotel n't 4* level.positives large bathroom mediterranean suite comfortable bed pillowsattentive housekeeping staffnegatives ac unit malfunctioned stay desk disorganized, missed 3 separate wakeup calls, concierge busy hard touch, did n't provide guidance special requests.tv hard use ipod sound dock suite non functioning. decided book mediterranean suite 3 night weekend stay 1st choice rest party filled, comparison w spent 45 night larger square footage room great soaking tub whirlpool jets nice shower.before stay hotel arrange car service price 53 tip reasonable driver waiting arrival.checkin easy downside room picked 2 person jacuzi tub no bath accessories salts bubble bath did n't stay, night got 12/1a checked voucher bottle champagne nice gesture fish waiting room, impression room huge open space felt room big, tv far away bed chore change channel, ipod dock broken disappointing.in morning way asked desk check thermostat said 65f 74 2 degrees warm try cover face night bright blue light kept, got room night no, 1st drop desk, called maintainence came look thermostat told play settings happy digital box wo n't work, asked wakeup 10am morning did n't happen, called later 6pm nap wakeup forgot, 10am wakeup morning yep forgotten.the bathroom facilities great room surprised room sold whirlpool bath tub n't bath amenities, great relax water jets going,",3
3,"unique, great stay, wonderful time hotel monaco, location excellent short stroll main downtown shopping area, pet friendly room showed no signs animal hair smells, monaco suite sleeping area big striped curtains pulled closed nice touch felt cosy, goldfish n

In [11]:
data['Rating'].unique()

array([4, 2, 3, 5, 1])

In [12]:
data['Rating'].value_counts()

5    9054
4    6039
3    2184
2    1793
1    1421
Name: Rating, dtype: int64

# Data Pre-processing:
- Remove html tags.
- Replace contractions in string. (e.g. replace I'm --> I am) and so on.\
- Remove numbers.
- Tokenization
- To remove Stopwords.
- Lemmatized data
- We have used NLTK library to tokenize words , remove stopwords and lemmatize the remaining words.

In [13]:
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

data['Review'] = data['Review'].apply(lambda x: strip_html(x))
data.head()

,Review,Rating
0,"nice hotel expensive parking got good deal stay hotel anniversary, arrived late evening took advice previous reviews did valet parking, check quick easy, little disappointed non-existent view room room clean nice size, bed comfortable woke stiff neck high pillows, not soundproof like heard music room night morning loud bangs doors opening closing hear people talking hallway, maybe just noisy neighbors, aveda bath products nice, did not goldfish stay nice touch taken advantage staying longer, location great walking distance shopping, overall nice experience having pay 40 parking night,",4
1,"ok nothing special charge diamond member hilton decided chain shot 20th anniversary seattle, start booked suite paid extra website description not, suite bedroom bathroom standard hotel room, took printed reservation desk showed said things like tv couch ect desk clerk told oh mixed suites description kimpton website sorry free breakfast, got kidding, embassy suits sitting room bathroom bedroom unlike kimpton calls suite, 5 day stay offer correct false advertising, send kimpton preferred guest website email asking failure provide suite advertised website reservation description furnished hard copy reservation printout website desk manager duty did not reply solution, send email trip guest survey did not follow email mail, guess tell concerned guest.the staff ranged indifferent not helpful, asked desk good breakfast spots neighborhood hood told no hotels, gee best breakfast spots seattle 1/2 block away convenient hotel does not know exist, arrived late night 11 pm inside run bellman busy chating cell phone help bags.prior arrival emailed hotel inform 20th anniversary half really picky wanted make sure good, got nice email saying like deliver bottle champagne chocolate covered strawberries room arrival celebrate, told needed foam pillows, arrival no champagne strawberries no foam pillows great room view alley high rise building good not better housekeeping staff cleaner room property, impressed left morning shopping room got short trips 2 hours, beds comfortable.not good ac-heat control 4 x 4 inch screen bring green shine directly eyes night, light sensitive tape controls.this not 4 start hotel clean business hotel super high rates, better chain hotels seattle,",2
2,"nice rooms not 4* experience hotel monaco seattle good hotel n't 4* level.positives large bathroom mediterranean suite comfortable bed pillowsattentive housekeeping staffnegatives ac unit malfunctioned stay desk disorganized, missed 3 separate wakeup calls, concierge busy hard touch, did n't provide guidance special requests.tv hard use ipod sound dock suite non functioning. decided book mediterranean suite 3 night weekend stay 1st choice rest party filled, comparison w spent 45 night larger square footage room great soaking tub whirlpool jets nice shower.before stay hotel arrange car service price 53 tip reasonable driver waiting arrival.checkin easy downside room picked 2 person jacuzi tub no bath accessories salts bubble bath did n't stay, night got 12/1a checked voucher bottle champagne nice gesture fish waiting room, impression room huge open space felt room big, tv far away bed chore change channel, ipod dock broken disappointing.in morning way asked desk check thermostat said 65f 74 2 degrees warm try cover face night bright blue light kept, got room night no, 1st drop desk, called maintainence came look thermostat told play settings happy digital box wo n't work, asked wakeup 10am morning did n't happen, called later 6pm nap wakeup forgot, 10am wakeup morning yep forgotten.the bathroom facilities great room surprised room sold whirlpool bath tub n't bath amenities, great relax water jets going,",3
3,"unique, great stay, wonderful time hotel monaco, location excellent short stroll main downtown shopping area, pet friendly room showed no signs animal hair smells, monaco suite sleeping area big striped curtains pulled closed nice touch felt cosy, goldfish n

In [14]:
def replace_contractions(text):
    """Replace contractions in string of Review"""
    return contractions.fix(text)

data['Review'] = data['Review'].apply(lambda x: replace_contractions(x))
data.head()

,Review,Rating
0,"nice hotel expensive parking got good deal stay hotel anniversary, arrived late evening took advice previous reviews did valet parking, check quick easy, little disappointed non-existent view room room clean nice size, bed comfortable woke stiff neck high pillows, not soundproof like heard music room night morning loud bangs doors opening closing hear people talking hallway, maybe just noisy neighbors, aveda bath products nice, did not goldfish stay nice touch taken advantage staying longer, location great walking distance shopping, overall nice experience having pay 40 parking night,",4
1,"ok nothing special charge diamond member hilton decided chain shot 20th anniversary seattle, start booked suite paid extra website description not, suite bedroom bathroom standard hotel room, took printed reservation desk showed said things like tv couch ect desk clerk told oh mixed suites description kimpton website sorry free breakfast, got kidding, embassy suits sitting room bathroom bedroom unlike kimpton calls suite, 5 day stay offer correct false advertising, send kimpton preferred guest website email asking failure provide suite advertised website reservation description furnished hard copy reservation printout website desk manager duty did not reply solution, send email trip guest survey did not follow email mail, guess tell concerned guest.the staff ranged indifferent not helpful, asked desk good breakfast spots neighborhood hood told no hotels, gee best breakfast spots seattle 1/2 block away convenient hotel does not know exist, arrived late night 11 pm inside run bellman busy chating cell phone help bags.prior arrival emailed hotel inform 20th anniversary half really picky wanted make sure good, got nice email saying like deliver bottle champagne chocolate covered strawberries room arrival celebrate, told needed foam pillows, arrival no champagne strawberries no foam pillows great room view alley high rise building good not better housekeeping staff cleaner room property, impressed left morning shopping room got short trips 2 hours, beds comfortable.not good ac-heat control 4 x 4 inch screen bring green shine directly eyes night, light sensitive tape controls.this not 4 start hotel clean business hotel super high rates, better chain hotels seattle,",2
2,"nice rooms not 4* experience hotel monaco seattle good hotel n't 4* level.positives large bathroom mediterranean suite comfortable bed pillowsattentive housekeeping staffnegatives ac unit malfunctioned stay desk disorganized, missed 3 separate wakeup calls, concierge busy hard touch, did n't provide guidance special requests.tv hard use ipod sound dock suite non functioning. decided book mediterranean suite 3 night weekend stay 1st choice rest party filled, comparison w spent 45 night larger square footage room great soaking tub whirlpool jets nice shower.before stay hotel arrange car service price 53 tip reasonable driver waiting arrival.checkin easy downside room picked 2 person jacuzi tub no bath accessories salts bubble bath did n't stay, night got 12/1a checked voucher bottle champagne nice gesture fish waiting room, impression room huge open space felt room big, tv far away bed chore change channel, ipod dock broken disappointing.in morning way asked desk check thermostat said 65f 74 2 degrees warm try cover face night bright blue light kept, got room night no, 1st drop desk, called maintainence came look thermostat told play settings happy digital box wo n't work, asked wakeup 10am morning did n't happen, called later 6pm nap wakeup forgot, 10am wakeup morning yep forgotten.the bathroom facilities great room surprised room sold whirlpool bath tub n't bath amenities, great relax water jets going,",3
3,"unique, great stay, wonderful time hotel monaco, location excellent short stroll main downtown shopping area, pet friendly room showed no signs animal hair smells, monaco suite sleeping area big striped curtains pulled closed nice touch felt cosy, goldfish n

In [15]:
# Tokenization of data
data['Review'] = data.apply(lambda row: nltk.word_tokenize(row['Review']), axis=1) 

In [16]:
data.head()

,Review,Rating
0,"[nice, hotel, expensive, parking, got, good, deal, stay, hotel, anniversary, ,, arrived, late, evening, took, advice, previous, reviews, did, valet, parking, ,, check, quick, easy, ,, little, disappointed, non-existent, view, room, room, clean, nice, size, ,, bed, comfortable, woke, stiff, neck, high, pillows, ,, not, soundproof, like, heard, music, room, night, morning, loud, bangs, doors, opening, closing, hear, people, talking, hallway, ,, maybe, just, noisy, neighbors, ,, aveda, bath, products, nice, ,, did, not, goldfish, stay, nice, touch, taken, advantage, staying, longer, ,, location, great, walking, distance, shopping, ,, overall, nice, experience, having, pay, 40, parking, night, ,]",4
1,"[ok, nothing, special, charge, diamond, member, hilton, decided, chain, shot, 20th, anniversary, seattle, ,, start, booked, suite, paid, extra, website, description, not, ,, suite, bedroom, bathroom, standard, hotel, room, ,, took, printed, reservation, desk, showed, said, things, like, tv, couch, ect, desk, clerk, told, oh, mixed, suites, description, kimpton, website, sorry, free, breakfast, ,, got, kidding, ,, embassy, suits, sitting, room, bathroom, bedroom, unlike, kimpton, calls, suite, ,, 5, day, stay, offer, correct, false, advertising, ,, send, kimpton, preferred, guest, website, email, asking, failure, provide, suite, advertised, website, reservation, description, furnished, hard, copy, reservation, printout, website, desk, manager, duty, did, ...]",2
2,"[nice, rooms, not, 4, *, experience, hotel, monaco, seattle, good, hotel, n't, 4, *, level.positives, large, bathroom, mediterranean, suite, comfortable, bed, pillowsattentive, housekeeping, staffnegatives, ac, unit, malfunctioned, stay, desk, disorganized, ,, missed, 3, separate, wakeup, calls, ,, concierge, busy, hard, touch, ,, did, n't, provide, guidance, special, requests.tv, hard, use, ipod, sound, dock, suite, non, functioning, ., decided, book, mediterranean, suite, 3, night, weekend, stay, 1st, choice, rest, party, filled, ,, comparison, w, spent, 45, night, larger, square, footage, room, great, soaking, tub, whirlpool, jets, nice, shower.before, stay, hotel, arrange, car, service, price, 53, tip, reasonable, driver, waiting, arrival.checkin, easy, ...]",3
3,"[unique, ,, great, stay, ,, wonderful, time, hotel, monaco, ,, location, excellent, short, stroll, main, downtown, shopping, area, ,, pet, friendly, room, showed, no, signs, animal, hair, smells, ,, monaco, suite, sleeping, area, big, striped, curtains, pulled, closed, nice, touch, felt, cosy, ,, goldfish, named, brandi, enjoyed, ,, did, n't, partake, free, wine, coffee/tea, service, lobby, thought, great, feature, ,, great, staff, friendly, ,, free, wireless, internet, hotel, worked, suite, 2, laptops, ,, decor, lovely, eclectic, mix, pattens, color, palatte, ,, animal, print, bathrobes, feel, like, rock, stars, ,, nice, did, n't, look, like, sterile, chain, hotel, hotel, personality, excellent, ...]",5
4,"[great, stay, great, stay, ,, went, seahawk, game, awesome, ,, downfall, view, building, did, n't, complain, ,, room, huge, staff, helpful, ,, booked, hotels, website, seahawk, package, ,, no, charge, parking, got, voucher, taxi, ,, problem, taxi, driver, did, n't, want, accept, voucher, barely, spoke, english, ,, funny, thing, speak, arabic, called, started, making, comments, girlfriend, cell, phone, buddy, ,, took, second, realize, just, said, fact, speak, language, face, priceless, ,, ass, told, ,, said, large, city, ,, told, head, doorman, issue, called, cab, company, promply, answer, did, n't, ,, apologized, offered, pay, taxi, ,, bucks, 2, miles, stadium, ,, ...]",5


In [21]:
# stopwords = stopwords.words('english')

customlist = ['not', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn',
        "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',
        "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn',
        "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

# Set custom stop-word's list as not, couldn't etc. words matter in Sentiment, so not removing them from original data.

stopwords = list(set(stopwords.words('english')) - set(customlist))  

# 2. Remove punctuations, special characters and stopwords from the Review column. Convert the Review to lower case.

In [22]:
lemmatizer = WordNetLemmatizer()

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in stopwords:
            new_words.append(word)
    return new_words

def lemmatize_list(words):
    new_words = []
    for word in words:
      new_words.append(lemmatizer.lemmatize(word, pos='v'))
    return new_words

def normalize(words):
    words = remove_non_ascii(words)
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_stopwords(words)
    words = lemmatize_list(words)
    return ' '.join(words)

data['Review'] = data.apply(lambda row: normalize(row['Review']), axis=1)
data.head()

,Review,Rating
0,nice hotel expensive park get good deal stay hotel anniversary arrive late even take advice previous review valet park check quick easy little disappoint nonexistent view room room clean nice size bed comfortable wake stiff neck high pillow not soundproof like hear music room night morning loud bang doors open close hear people talk hallway maybe noisy neighbor aveda bath products nice not goldfish stay nice touch take advantage stay longer location great walk distance shop overall nice experience pay 40 park night,4
1,ok nothing special charge diamond member hilton decide chain shoot 20th anniversary seattle start book suite pay extra website description not suite bedroom bathroom standard hotel room take print reservation desk show say things like tv couch ect desk clerk tell oh mix suit description kimpton website sorry free breakfast get kid embassy suit sit room bathroom bedroom unlike kimpton call suite 5 day stay offer correct false advertise send kimpton prefer guest website email ask failure provide suite advertise website reservation description furnish hard copy reservation printout website desk manager duty not reply solution send email trip guest survey not follow email mail guess tell concern guestthe staff range indifferent not helpful ask desk good breakfast spot neighborhood hood tell hotels gee best breakfast spot seattle 12 block away convenient hotel not know exist arrive late night 11 pm inside run bellman busy chat cell phone help bagsprior arrival email hotel inform 20th anniversary half really picky want make sure good get nice email say like deliver bottle champagne chocolate cover strawberries room arrival celebrate tell need foam pillow arrival champagne strawberries foam pillow great room view alley high rise build good not better housekeep staff cleaner room property impress leave morning shop room get short trip 2 hours bed comfortablenot good acheat control 4 x 4 inch screen bring green shine directly eye night light sensitive tape controlsthis not 4 start hotel clean business hotel super high rat better chain hotels seattle,2
2,nice room not 4 experience hotel monaco seattle good hotel nt 4 levelpositives large bathroom mediterranean suite comfortable bed pillowsattentive housekeep staffnegatives ac unit malfunction stay desk disorganize miss 3 separate wakeup call concierge busy hard touch nt provide guidance special requeststv hard use ipod sound dock suite non function decide book mediterranean suite 3 night weekend stay 1st choice rest party fill comparison w spend 45 night larger square footage room great soak tub whirlpool jet nice showerbefore stay hotel arrange car service price 53 tip reasonable driver wait arrivalcheckin easy downside room pick 2 person jacuzi tub bath accessories salt bubble bath nt stay night get 121a check voucher bottle champagne nice gesture fish wait room impression room huge open space felt room big tv far away bed chore change channel ipod dock break disappointingin morning way ask desk check thermostat say 65f 74 2 degrees warm try cover face night bright blue light keep get room night 1st drop desk call maintainence come look thermostat tell play settings happy digital box wo nt work ask wakeup 10am morning nt happen call later 6pm nap wakeup forget 10am wakeup morning yep forgottenthe bathroom facilities great room surprise room sell whirlpool bath tub nt bath amenities great relax water jet go,3
3,unique great stay wonderful time hotel monaco location excellent short stroll main downtown shop area pet friendly room show sign animal hair smell monaco suite sleep area big strip curtain pull close nice touch felt cosy goldfish name brandi enjoy nt partake free wine coffeetea service lobby think great feature great staff friendly free wireless internet hotel work suite 2 laptops decor lovely eclectic mix pattens color palatte animal print bathrobes feel like rock star nice nt look like sterile chain hotel hotel personality excellent stay,5
4,gr

# 3-1. Create two objects X and y. X will be the 'Review' column dataframe and y will be the “Rating” column. create a CountVectorizer object and split the data into training and testing sets. 

In [73]:
# Vectorization (Convert text data to numbers).
vectorizer = CountVectorizer(max_features=5000)
data_features = vectorizer.fit_transform(data['Review'])

data_features = data_features.toarray()  

In [74]:
data_features.shape

(20491, 5000)

In [75]:
X = data_features
y = data.Rating

In [76]:
# Split data into training and testing set.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3-2. Train a MultinomialNB model and Display the confusion Matrix

In [77]:
# training a Naive Bayes classifier 
mnb = MultinomialNB().fit(X_train, y_train) 

y_pred_NB=mnb.predict(X_test)

print("Accuracy of Multinominal Naive Balyes:",accuracy_score(y_test, y_pred_NB))
print(classification_report(y_pred_NB,y_test))

Accuracy of Multinominal Naive Balyes: 0.6062454257135886
              precision    recall  f1-score   support

           1       0.65      0.67      0.66       284
           2       0.45      0.39      0.42       380
           3       0.34      0.36      0.35       414
           4       0.42      0.54      0.48       983
           5       0.82      0.72      0.77      2038

    accuracy                           0.61      4099
   macro avg       0.54      0.54      0.53      4099
weighted avg       0.63      0.61      0.61      4099



In [78]:
metrics.confusion_matrix(y_test,y_pred_NB)

array([[ 191,   66,   23,    9,    3],
       [  58,  149,   98,   19,    9],
       [  20,   79,  149,  154,   30],
       [  10,   59,  121,  531,  531],
       [   5,   27,   23,  270, 1465]])

# 4. Display the POS tagging on the first 4 rows of ‘Review

In [79]:
data.head(4)

,Review,Rating
0,nice hotel expensive park get good deal stay hotel anniversary arrive late even take advice previous review valet park check quick easy little disappoint nonexistent view room room clean nice size bed comfortable wake stiff neck high pillow not soundproof like hear music room night morning loud bang doors open close hear people talk hallway maybe noisy neighbor aveda bath products nice not goldfish stay nice touch take advantage stay longer location great walk distance shop overall nice experience pay 40 park night,4
1,ok nothing special charge diamond member hilton decide chain shoot 20th anniversary seattle start book suite pay extra website description not suite bedroom bathroom standard hotel room take print reservation desk show say things like tv couch ect desk clerk tell oh mix suit description kimpton website sorry free breakfast get kid embassy suit sit room bathroom bedroom unlike kimpton call suite 5 day stay offer correct false advertise send kimpton prefer guest website email ask failure provide suite advertise website reservation description furnish hard copy reservation printout website desk manager duty not reply solution send email trip guest survey not follow email mail guess tell concern guestthe staff range indifferent not helpful ask desk good breakfast spot neighborhood hood tell hotels gee best breakfast spot seattle 12 block away convenient hotel not know exist arrive late night 11 pm inside run bellman busy chat cell phone help bagsprior arrival email hotel inform 20th anniversary half really picky want make sure good get nice email say like deliver bottle champagne chocolate cover strawberries room arrival celebrate tell need foam pillow arrival champagne strawberries foam pillow great room view alley high rise build good not better housekeep staff cleaner room property impress leave morning shop room get short trip 2 hours bed comfortablenot good acheat control 4 x 4 inch screen bring green shine directly eye night light sensitive tape controlsthis not 4 start hotel clean business hotel super high rat better chain hotels seattle,2
2,nice room not 4 experience hotel monaco seattle good hotel nt 4 levelpositives large bathroom mediterranean suite comfortable bed pillowsattentive housekeep staffnegatives ac unit malfunction stay desk disorganize miss 3 separate wakeup call concierge busy hard touch nt provide guidance special requeststv hard use ipod sound dock suite non function decide book mediterranean suite 3 night weekend stay 1st choice rest party fill comparison w spend 45 night larger square footage room great soak tub whirlpool jet nice showerbefore stay hotel arrange car service price 53 tip reasonable driver wait arrivalcheckin easy downside room pick 2 person jacuzi tub bath accessories salt bubble bath nt stay night get 121a check voucher bottle champagne nice gesture fish wait room impression room huge open space felt room big tv far away bed chore change channel ipod dock break disappointingin morning way ask desk check thermostat say 65f 74 2 degrees warm try cover face night bright blue light keep get room night 1st drop desk call maintainence come look thermostat tell play settings happy digital box wo nt work ask wakeup 10am morning nt happen call later 6pm nap wakeup forget 10am wakeup morning yep forgottenthe bathroom facilities great room surprise room sell whirlpool bath tub nt bath amenities great relax water jet go,3
3,unique great stay wonderful time hotel monaco location excellent short stroll main downtown shop area pet friendly room show sign animal hair smell monaco suite sleep area big strip curtain pull close nice touch felt cosy goldfish name brandi enjoy nt partake free wine coffeetea service lobby think great feature great staff friendly free wireless internet hotel work suite 2 laptops decor lovely eclectic mix pattens color palatte animal print bathrobes feel like rock star nice nt look like sterile chain hotel hotel personality excellent stay,5


In [80]:
reviews = data['Review'].head(4).tolist()
>>> tagged_reviews = pos_tag_sents(map(word_tokenize, reviews))
>>> tagged_reviews

[[('nice', 'JJ'),
  ('hotel', 'NN'),
  ('expensive', 'JJ'),
  ('park', 'NN'),
  ('get', 'VB'),
  ('good', 'JJ'),
  ('deal', 'NN'),
  ('stay', 'VB'),
  ('hotel', 'NN'),
  ('anniversary', 'JJ'),
  ('arrive', 'NN'),
  ('late', 'JJ'),
  ('even', 'RB'),
  ('take', 'VBP'),
  ('advice', 'NN'),
  ('previous', 'JJ'),
  ('review', 'NN'),
  ('valet', 'NN'),
  ('park', 'NN'),
  ('check', 'NN'),
  ('quick', 'JJ'),
  ('easy', 'JJ'),
  ('little', 'JJ'),
  ('disappoint', 'JJ'),
  ('nonexistent', 'JJ'),
  ('view', 'NN'),
  ('room', 'NN'),
  ('room', 'NN'),
  ('clean', 'JJ'),
  ('nice', 'JJ'),
  ('size', 'NN'),
  ('bed', 'VBD'),
  ('comfortable', 'JJ'),
  ('wake', 'NN'),
  ('stiff', 'JJ'),
  ('neck', 'RB'),
  ('high', 'JJ'),
  ('pillow', 'NN'),
  ('not', 'RB'),
  ('soundproof', 'JJ'),
  ('like', 'IN'),
  ('hear', 'JJ'),
  ('music', 'NN'),
  ('room', 'NN'),
  ('night', 'NN'),
  ('morning', 'NN'),
  ('loud', 'NN'),
  ('bang', 'NN'),
  ('doors', 'NNS'),
  ('open', 'VBP'),
  ('close', 'JJ'),
  ('hear', 'JJ'

# 5. Build and display a dependency parser tree for the sentence:
# "He is last seen being taken inside Castle Black for treatment while yelling for his men to continue fighting."

In [81]:
!pip install spacy

In [82]:
!python -m spacy download en_core_web_lg 

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.1.0/en_core_web_lg-3.1.0-py3-none-any.whl (777.1 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


In [83]:
import spacy

nlp = spacy.load('en_core_web_lg')
text=("He is last seen being taken inside Castle Black for treatment while yelling for his men to continue fighting.")
doc = nlp(text)

In [84]:
# Analyze syntax
print("Noun phrases:", [chunk.text for chunk in doc.noun_chunks])
print("Verbs:", [token.lemma_ for token in doc if token.pos_ == "VERB"])

Noun phrases: ['He', 'Castle Black', 'treatment', 'his men']
Verbs: ['see', 'take', 'yell', 'continue', 'fight']


In [85]:
for token in nlp(text):
 print(token.text,'=>',token.dep_,'=>',token.head.text)

He => nsubjpass => seen
is => auxpass => seen
last => advmod => seen
seen => ROOT => seen
being => auxpass => taken
taken => xcomp => seen
inside => prep => taken
Castle => compound => Black
Black => pobj => inside
for => prep => taken
treatment => pobj => for
while => mark => yelling
yelling => advcl => taken
for => mark => continue
his => poss => men
men => nsubj => continue
to => aux => continue
continue => advcl => yelling
fighting => xcomp => continue
. => punct => seen


In [86]:
from spacy import displacy
displacy.render(nlp(text),jupyter=True)